In [ ]:
from sagemaker.pytorch import PyTorch
from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::359077243653:role/service-role/AmazonSageMaker-ExecutionRole-20251115T201648"
hyperparams = dict(
    device="cpu",
    epochs=1,
    batch_size=4,
    block_size=128,
    total_batch_size=8192,
    total_iters=500,
    eval_iters=100,
    eval_check=50,
    save_checkpoint_iter=1000,
    hf_token="...",  # if needed
)

estimator = PyTorch(
    entry_point="trainer.py",
    source_dir="...",
    role=role,
    instance_type="ml.c5.2xlarge",
    instance_count=1,
    framework_version="2.2",
    py_version="py310",
    hyperparameters=hyperparams,
    environment={"WANDB_DISABLED": "1"},
)
estimator.fit({"training": "..."})


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2025-11-18-09-18-47-395


2025-11-18 09:18:49 Starting - Starting the training job.

In [1]:
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 196.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 29.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 167.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 33.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [transformers] [transformers]ub]


In [2]:
# Avoid the cached sdist, pick a wheel-only version
!pip install --no-cache-dir "sentencepiece==0.1.99"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 173.4 MB/s  0:00:00


In [6]:
ckpt

{'MODEL_STATE': OrderedDict([('encoder.positional_embeddings_src.pe',
               tensor([[[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
                          0.0000e+00,  1.0000e+00],
                        [ 8.4147e-01,  5.4030e-01,  8.2186e-01,  ...,  1.0000e+00,
                          1.0366e-04,  1.0000e+00],
                        [ 9.0930e-01, -4.1615e-01,  9.3641e-01,  ...,  1.0000e+00,
                          2.0733e-04,  1.0000e+00],
                        ...,
                        [ 6.1950e-02,  9.9808e-01,  7.9820e-01,  ...,  9.9850e-01,
                          5.2740e-02,  9.9861e-01],
                        [ 8.7333e-01,  4.8714e-01,  9.4981e-01,  ...,  9.9850e-01,
                          5.2844e-02,  9.9860e-01],
                        [ 8.8177e-01, -4.7168e-01,  2.8401e-01,  ...,  9.9849e-01,
                          5.2947e-02,  9.9860e-01]]])),
              ('encoder.src_text_embeds.embeddings_table.weight',
               tensor

In [1]:
import torch
from model import Transformer
from tokenizer import initialize_tokenizer
from config import ModelArgs
from inference import topk_sampling  # or whatever helper you want

ckpt = torch.load("snapshot_249.pt", map_location="cpu")

args = ModelArgs(
    block_size=128,            # match what you trained with
    batch_size=4,
    device="cpu",
)
tokenizer = initialize_tokenizer(args.hf_token)
args.src_vocab_size = args.tgt_vocab_size = len(tokenizer)

model = Transformer(
    src_vocab_size=args.src_vocab_size,
    tgt_vocab_size=args.tgt_vocab_size,
    use_liger=args.use_liger,
)
model.load_state_dict(ckpt["MODEL_STATE"])
model.eval()


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


Liger kernel not available, using standard cross entropy


Transformer(
  (encoder): EncoderModel(
    (positional_embeddings_src): SinusoidalPositionalEmbeddings()
    (src_text_embeds): SrcTextEmbeddings(
      (embeddings_table): Embedding(64015, 512)
    )
    (encoder_layers): ModuleList(
      (0-5): 6 x TransformerEncoderBlock(
        (mha): FullMHA(
          (heads): ModuleList(
            (0-7): 8 x FullAttentionHead(
              (query): Linear(in_features=512, out_features=64, bias=False)
              (keys): Linear(in_features=512, out_features=64, bias=False)
              (values): Linear(in_features=512, out_features=64, bias=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (dropout): Dropout(p=0.1, inplace=False)
          (linear): Linear(in_features=512, out_features=512, bias=False)
        )
        (layer_norm1): LayerNormalization(
          (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (layer_norm2): LayerNormalization(
          (no

In [2]:
prompt = "Hello, how are you?"
translation = topk_sampling(model, prompt, tokenizer, device="cpu", max_length=50)
print("Prompt:", prompt)
print("Translation:", translation)


Input IDs:  torch.Size([1, 50])
Target IDs:  torch.Size([1, 19])
Prompt: Hello, how are you?
Translation: [CLS] उन्होंने मेंी में ही से पर की के में की केना पर की है है[SEP]


In [6]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.2 MB/s  0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached aiohttp-3.13.2-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (8.1 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached async_timeout-5.0.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached frozenlist-1.8.0-cp310-cp310-manylinux1_x86_64.manylinux_2_28_x86_64.manylinux_2_5_x86_64.whl.metadata (20 kB)
  Using cached multidict-6.7.0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (5.3 kB)
  Using cached propcache-0.4.1-cp310-cp310-manylinux2014_x86_64.many

In [10]:

!pip install --no-cache-dir "datasets>=2.15.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 114.1 MB/s  0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 196.1 MB/s  0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for pyarrow (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [829 lines of output]
      /tmp/pip-build-env-ptcx6dw4/overlay/lib/python3.10/site-packages/setuptools/config/_apply_pyprojecttoml.py:82: SetuptoolsDeprecationWarning: `project.license` as a TOML table is deprecated
      !!
      
              ********************************************************************************
              Please use a simple string containing a SPDX expression for `project.license`. You can also use `project.license-files`. (Both options available on setuptools>=77.0.0).
      
              By 2026-Feb-18, you need to update your pro

ModuleNotFoundError: No module named 'datasets'